In [1]:
class Node:
    def __init__(n, x, y, g, h, parent=None): #constructor
        n.x = x
        n.y = y
        n.g = g  # g(n) = cost from start node to current node 'n'
        n.h = h  # h(n) = heuristic estimate from current node 'n' to goal node
        n.parent = parent  # its parent node

    def f(n):
        return n.g + n.h  # f(n) = g(n) + h(n)

def heuristic(node, goal):
    # using the Manhattan Distance heuristic
    return abs(node.x - goal.x) + abs(node.y - goal.y)

def is_valid_move(cube, x, y):
    return 0 <= x < len(cube) and 0 <= y < len(cube[0]) and cube[x][y] != 1

def get_neighbors(cube, n):
    neighbors = []
    for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        new_x, new_y = n.x + dx, n.y + dy
        if is_valid_move(cube, new_x, new_y):
            neighbors.append(Node(new_x, new_y, 0, 0))
    return neighbors

def A_star(cube, start, goal):
    start_node = Node(start.x, start.y, 0, heuristic(Node(start.x, start.y, 0, 0), goal))
    open = [start_node]
    closed = set()

    while open:
        current_node = min(open, key=lambda node: node.f()) # node with the smallest f value in the open list.
        open.remove(current_node)
        closed.add((current_node.x, current_node.y))

        if (current_node.x, current_node.y) == (goal.x, goal.y):
            path = []
            while current_node is not None:
                path.append((current_node.x, current_node.y))
                current_node = current_node.parent
            path.reverse()
            return path

        for neighbor in get_neighbors(cube, current_node):
            if (neighbor.x, neighbor.y) in closed:
                continue
            temp_g = current_node.g + 1
            if cube[neighbor.x][neighbor.y] == 2:  # if the neighbor is a short wall, add extra cost
                temp_g += 10
            if neighbor not in open or temp_g < neighbor.g:
                neighbor.g = temp_g
                neighbor.h = heuristic(neighbor, goal)
                neighbor.parent = current_node
                if neighbor not in open:
                    open.append(neighbor)
    return None

def readFile(filename):
  cube = []
  with open(filename, 'r') as f:
    for line in f:
      row = [int(char) for char in line.strip('\n')] # removing the newlines using strip()
      cube.append(row)
  return cube
  
def print_maze_with_path(maze, path):
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if (i, j) == path[0]:
                print("S", end=" ")
            elif (i, j) == path[-1]:
                print("G", end=" ")
            elif (i, j) in path:
                print("*", end=" ")
            elif maze[i][j] == 0:
                print("0", end=" ")
            elif maze[i][j] == 1:
                print("1", end=" ")
            elif maze[i][j] == 2:
                print("2", end=" ")
        print()

# main
cube = [
    [1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 1, 1],
    [0, 1, 0, 1, 2, 0, 0],
    [1, 1, 0, 1, 1, 2, 1],
    [0, 1, 0, 2, 0, 2, 0],
    [0, 1, 1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0]
]
# 2 represents the short wall that we can jump over
# 1 represents the wall that we cannot pass through
# 0 represents the path that we can walk through

start = Node(0, 2, 0, 0) # initialises the x, y, g, h parameters
goal = Node(6, 0, 0, 0)

path = A_star(cube, start, goal)
if path:
    print_maze_with_path(cube, path)
    print("Shortest path:", path)
else:
    print("No path found.")

1 0 S 0 1 0 0 
1 1 * 0 0 1 1 
0 1 * 1 2 0 0 
1 1 * 1 1 2 1 
0 1 * * * 2 0 
0 1 1 1 * 1 1 
G * * * * 0 0 
Shortest path: [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]


In [1]:
"""
Tic Tac Toe Player
"""
import copy


X = "X"
O = "O"
EMPTY = None


def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]
    
def draw_board(board):
    print("-------------")
    for i in range(3):
        print("| ", end="")
        for j in range(3):
            if board[i][j] == EMPTY:
                print(" ", end="")
            else:
                print(board[i][j], end="")
            print(" | ", end="")
        print()
        print("-------------")


def player(board):
    """
    Returns player who has the next turn on a board.
    """
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1
    if count % 2 != 0:
        return O
    return X


def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res


def result(board, action, x):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board) # leaving the original untouched until we decide on the best move
    (i, j) = action
    
    if x == 0:
        result_board[i][j] = EMPTY
    else: 
        result_board[i][j] = curr_player
    
    return result_board


def get_horizontal_winner(board):
    # check horizontally
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[i][0]
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val
    return winner_val


def get_vertical_winner(board):
    # check vertically
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[0][i]
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val
    return winner_val


def get_diagonal_winner(board):
    # check diagonally
    winner_val = None
    board_len = len(board)
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
    if winner_val:
        return winner_val

    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None

    return winner_val


def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
    return winner_val


def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) != None:
        return True

    for i in board:
        for j in i:
            if j == EMPTY:
                return False
    return True

def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

def isValidMove(board, row, col):
    # Check if the move is within the bounds of the board
    if row < 3 and col < 3 and row >= 0 and col >= 0:
        # Check if the position is not already occupied
        if board[row][col] is EMPTY:
            return True
    return False

def alpha_beta_pruning(board, alpha, beta, maxPlayer):

    #Returns the optimal action for the current player on the board.
    row = -1
    col = -1

    if terminal(board):
        return [row, col, utility(board)]


    else:
        for move in actions(board):
            board = result(board, move, maxPlayer)
            value = alpha_beta_pruning(board, alpha, beta, -maxPlayer)

            if maxPlayer == True:
                # X is always the max player
                if value[2] > alpha:
                    alpha = value[2]
                    row, col = move

            else:
                if value[2] < beta:
                    beta = value[2]
                    row, col = move

            board = result(board, move, 0)

            if alpha >= beta:
                break

        if maxPlayer == True:
            return [row, col, alpha]

        else:
            return [row, col, beta]
      
def minmax(board):
    row, col, value = alpha_beta_pruning(board, float('-inf'), float('inf'), -1)
    return (row, col)
    
        
def play_game():

    board = initial_state()
    draw_board(board)
    while not terminal(board):

        # player 1 turn
        print("Enter the row and column number (separated by space)")
        while(True):
            row, col = map(int, input().split())
            if isValidMove(board, row, col):
                break
            else:
                print("Invalid move\nEnter the row and column number again:")

        board = result(board, (row, col), 1)
        
        # player 2 turn
        if not terminal(board):
            (row, col) = minmax(board)
            board = result(board, (row, col), -1)
            draw_board(board)

    print("Winner is ", winner(board))
    
# main
play_game()

-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Enter the row and column number (separated by space)
-------------
|   | X |   | 
-------------
|   |   |   | 
-------------
|   | O |   | 
-------------
Enter the row and column number (separated by space)
-------------
| X | X | O | 
-------------
|   |   |   | 
-------------
|   | O |   | 
-------------
Enter the row and column number (separated by space)
-------------
| X | X | O | 
-------------
|   | X |   | 
-------------
|   | O | O | 
-------------
Enter the row and column number (separated by space)
-------------
| X | X | O | 
-------------
|   | X | O | 
-------------
| X | O | O | 
-------------
Winner is  O
